In [1]:
%pylab
%matplotlib inline

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
cd ..

/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-net-work


In [3]:
import sys
import numpy as np
import skimage
import cv2
import sklearn
import imp

In [4]:
import holoviews

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [24]:
import neukrill_net.utils
import neukrill_net.image_features
import neukrill_net.highlevelfeatures
import neukrill_net.stacked

In [17]:
import skimage.feature

In [21]:
import sklearn.ensemble

In [6]:
import time

In [7]:
#%pdb

In [8]:
settings = neukrill_net.utils.Settings('settings.json')

In [9]:
X,y = settings.flattened_train_paths(settings.classes)

In [13]:
reload(neukrill_net.highlevelfeatures)

<module 'neukrill_net.highlevelfeatures' from '/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-net-tools/neukrill_net/highlevelfeatures.py'>

In [25]:
reload(neukrill_net.image_features)

<module 'neukrill_net.image_features' from '/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-net-tools/neukrill_net/image_features.py'>

## Construct the full feature list

In [14]:
attrlst = ['height','width','numpixels','aspectratio','mean','std','stderr',
           'numwhite','propwhite','numnonwhite','propnonwhite','numblack','propblack','numbool','propbool']

hlf  = neukrill_net.highlevelfeatures.BasicAttributes(attrlst)
hlf += neukrill_net.highlevelfeatures.Haralick()
hlf += neukrill_net.highlevelfeatures.ThresholdAdjacency()
hlf += neukrill_net.highlevelfeatures.ContourMoments()
hlf += neukrill_net.highlevelfeatures.ContourHistogram()
hlf += neukrill_net.highlevelfeatures.CoocurProps()

In [18]:
hlf.preprocess_and_extract_image(neukrill_net.highlevelfeatures.loadimage(X[0]))

array([  6.60000000e+01,   5.90000000e+01,   3.89400000e+03,
         8.93939394e-01,   2.52098870e+02,   1.87598078e+01,
         3.00628681e-01,   3.29300000e+03,   8.45659990e-01,
         6.01000000e+02,   1.54340010e-01,   0.00000000e+00,
         0.00000000e+00,   3.86200000e+03,   9.91782229e-01,
         5.92909729e-01,   1.17428038e+02,   8.37376309e-01,
         3.60378916e+02,   8.37396275e-01,   5.04056774e+02,
         1.32408763e+03,   1.77634694e+00,   2.17408130e+00,
         2.42847737e-03,   1.54530917e+00,  -2.79362396e-01,
         7.10802264e-01,   8.44503234e-03,   8.02833026e+01,
         1.08540997e-01,   6.01075856e+00,   7.49707356e-03,
         9.91907566e-02,   5.62402684e+01,   7.00702220e-02,
         9.22398670e-02,   4.07591384e-05,   1.03418161e-01,
         5.27140192e-02,   4.48146178e-02,   9.27835052e-02,
         8.24742268e-02,   1.28865979e-01,   2.57731959e-01,
         2.21649485e-01,   1.23711340e-01,   5.67010309e-02,
         1.03092784e-02,

In [20]:
neukrill_net.highlevelfeatures.ContourHistogram().preprocess_and_extract_image(neukrill_net.highlevelfeatures.loadimage(X[0]))

array([[ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.01036269],
       [ 0.        ],
       [ 0.01554404],
       [ 0.        ],
       [ 0.00518135],
       [ 0.        ],
       [ 0.00518135],
       [ 0.        ],
       [ 0.01036269],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.01036269],
       [ 0.00518135],
       [ 0.00518135],
       [ 0

In [23]:
kprf_base = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_depth=25,
                                                            min_samples_leaf=20, n_jobs=12, random_state=42)

In [26]:
max_num_kp = 150

detector_list = [lambda image: neukrill_net.image_features.get_ORB_keypoints(image, n=max_num_kp, patchSize=9),
                 lambda image: neukrill_net.image_features.get_BRISK_keypoints(image, n=max_num_kp),
                 lambda image: neukrill_net.image_features.get_MSER_keypoints(image, n=max_num_kp)]

describer_list = [neukrill_net.image_features.get_ORB_descriptions,
                  neukrill_net.image_features.get_BRISK_descriptions,
                  neukrill_net.image_features.get_ORB_descriptions]

In [27]:
for index,detector in enumerate(detector_list):
    hlf += neukrill_net.highlevelfeatures.KeypointEnsembleClassifier(detector, describer_list[index], kprf_base,
                                                                     return_num_kp=True, summary_method='vote')

In [28]:
rf_clf = sklearn.ensemble.RandomForestClassifier(n_estimators=2500, max_depth=30,
                                                            min_samples_leaf=1, n_jobs=12, random_state=42)

In [30]:
reload(neukrill_net.stacked)

<module 'neukrill_net.stacked' from '/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-net-tools/neukrill_net/stacked.py'>

In [32]:
import sklearn.pipeline

In [33]:
selector = sklearn.feature_selection.SelectPercentile(sklearn.feature_selection.f_classif, percentile=33)

In [39]:
stack_pipe = sklearn.pipeline.Pipeline([('filter', selector), ('clf', rf_clf)])

stacked_clf = neukrill_net.stacked.StackedClassifier(hlf, stack_pipe, inner_prop=0.25, random_state=42)

## Random Forest, no hierarchy

In [37]:
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(X, y, test_size=0.5, random_state=42)

In [41]:
clf = stacked_clf

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(X, y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-venv-auto/lib/python2.7/site-packages/sklearn/utils/validation.py:332: UserWarning: StandardScaler assumes floating point values as input, got uint8
  "got %s" % (estimator, X.dtype))


KeyError: '<ipython-input-29-3d856375b90e>'

In [44]:
kp_hlf = neukrill_net.highlevelfeatures.KeypointEnsembleClassifier(detector_list[1], describer_list[1], kprf_base,
                                                                     return_num_kp=True, summary_method='vote')

In [45]:
kp_stacked_clf = neukrill_net.stacked.StackedClassifier(kp_hlf, stack_pipe, inner_prop=0.25, random_state=42)

In [46]:
clf = kp_stacked_clf

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(X, y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=10872.469008


/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-venv-auto/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:106: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


AttributeError: StackedClassifier instance has no attribute 'score'

In [ ]:
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

# BREAK

This is similar to just the Contour Moments and Haralick features

On reduced

In [22]:
my_X = X_new

clf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_depth=20, min_samples_leaf=5, n_jobs=12, random_state=42)

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(my_X), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=23.738656044
Accuracy=0.502505274262
Logloss=1.94713154469


Does slightly worse with fewer features.

Maybe it was too few?

In [23]:
my_X = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100).fit_transform(XF.squeeze(0), y)

clf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_depth=20, min_samples_leaf=5, n_jobs=12, random_state=42)

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(my_X), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=35.6933050156
Accuracy=0.529733649789
Logloss=1.86473985616


### Hierarchical classifier

In [24]:
import neukrill_net.taxonomy

In [29]:
reload(neukrill_net.stacked)

<module 'neukrill_net.stacked' from '/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-net-tools/neukrill_net/stacked.py'>

In [36]:
reload(neukrill_net.taxonomy)

<module 'neukrill_net.taxonomy' from '/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-net-tools/neukrill_net/taxonomy.py'>

In [37]:
neukrill_net.taxonomy.taxonomy

{'no_class': {'artifacts': {}, 'artifacts_edge': {}},
 'plankton': {'chaetognaths': {'chaetognath_non_sagitta': {},
   'chaetognath_other': {},
   'chaetognath_sagitta': {}},
  'chordate_type1': {},
  'crustaceans': {'amphipods': {},
   'copepods': {'calanoid': {'copepod_calanoid': {},
     'copepod_calanoid_eggs': {},
     'copepod_calanoid_eucalanus': {},
     'copepod_calanoid_flatheads': {},
     'copepod_calanoid_frillyAntennae': {},
     'copepod_calanoid_large': {},
     'copepod_calanoid_large_side_antennatucked': {},
     'copepod_calanoid_octomoms': {},
     'copepod_calanoid_small_longantennae': {},
     'copepod_other': {}},
    'cyclopoid_copepods': {'copepod_cyclopoid_copilia': {},
     'oithona': {'copepod_cyclopoid_oithona': {},
      'copepod_cyclopoid_oithona_eggs': {}}}},
   'crustacean_other': {},
   'shrimp_like': {'decapods_all': {'decapods': {},
     'shrimp_caridean': {},
     'shrimp_sergestidae': {},
     'shrimp_zoea': {}},
    'euphausiids_all_ages': {'eupha

In [38]:
settings.classes

[u'acantharia_protist',
 u'acantharia_protist_big_center',
 u'acantharia_protist_halo',
 u'amphipods',
 u'appendicularian_fritillaridae',
 u'appendicularian_slight_curve',
 u'appendicularian_s_shape',
 u'appendicularian_straight',
 u'artifacts',
 u'artifacts_edge',
 u'chaetognath_non_sagitta',
 u'chaetognath_other',
 u'chaetognath_sagitta',
 u'chordate_type1',
 u'copepod_calanoid',
 u'copepod_calanoid_eggs',
 u'copepod_calanoid_eucalanus',
 u'copepod_calanoid_flatheads',
 u'copepod_calanoid_frillyAntennae',
 u'copepod_calanoid_large',
 u'copepod_calanoid_large_side_antennatucked',
 u'copepod_calanoid_octomoms',
 u'copepod_calanoid_small_longantennae',
 u'copepod_cyclopoid_copilia',
 u'copepod_cyclopoid_oithona',
 u'copepod_cyclopoid_oithona_eggs',
 u'copepod_other',
 u'crustacean_other',
 u'ctenophore_cestid',
 u'ctenophore_cydippid_no_tentacles',
 u'ctenophore_cydippid_tentacles',
 u'ctenophore_lobate',
 u'decapods',
 u'detritus_blob',
 u'detritus_filamentous',
 u'detritus_other',
 u'

In [39]:
marked_taxonomy = neukrill_net.stacked.propagate_labels_to_leaves(neukrill_net.taxonomy.taxonomy, settings.classes)

In [40]:
marked_taxonomy

{'no_class': {'artifacts': 8, 'artifacts_edge': 9},
 'plankton': {'chaetognaths': {'chaetognath_non_sagitta': 10,
   'chaetognath_other': 11,
   'chaetognath_sagitta': 12},
  'chordate_type1': 13,
  'crustaceans': {'amphipods': 3,
   'copepods': {'calanoid': {'copepod_calanoid': 14,
     'copepod_calanoid_eggs': 15,
     'copepod_calanoid_eucalanus': 16,
     'copepod_calanoid_flatheads': 17,
     'copepod_calanoid_frillyAntennae': 18,
     'copepod_calanoid_large': 19,
     'copepod_calanoid_large_side_antennatucked': 20,
     'copepod_calanoid_octomoms': 21,
     'copepod_calanoid_small_longantennae': 22,
     'copepod_other': 26},
    'cyclopoid_copepods': {'copepod_cyclopoid_copilia': 23,
     'oithona': {'copepod_cyclopoid_oithona': 24,
      'copepod_cyclopoid_oithona_eggs': 25}}},
   'crustacean_other': 27,
   'shrimp_like': {'decapods_all': {'decapods': 32,
     'shrimp_caridean': 92,
     'shrimp_sergestidae': 94,
     'shrimp_zoea': 95},
    'euphausiids_all_ages': {'euphausi

In [41]:
base_clf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_depth=20, min_samples_leaf=5, n_jobs=12, random_state=42)

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_clf)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=70.759239912
Logloss=1.8479429741
Time=17.3513498306
Accuracy=0.530129219409


### With all the features left in

In [42]:
base_clf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_depth=20, min_samples_leaf=5, n_jobs=12, random_state=42)

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_clf)

X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0)), y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=87.1275148392
Logloss=1.91641586977
Time=20.142124176
Accuracy=0.526305379747


Try with a pipline to reduce the number of features at each level

In [44]:
import sklearn.pipeline

In [47]:
base_clf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_depth=20, min_samples_leaf=5, n_jobs=12, random_state=42)
best_filter = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100)

base_pipe = sklearn.pipeline.Pipeline([('filter', best_filter), ('clf', base_clf)])

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_pipe)

my_X = sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0))
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(my_X, y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=68.5872459412
Logloss=1.8722423589
Time=18.8261601925
Accuracy=0.520767405063


/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-venv-auto/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:106: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-venv-auto/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:106: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [47]:
base_clf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_depth=20, min_samples_leaf=5, n_jobs=12, random_state=42)
best_filter = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100)

base_pipe = sklearn.pipeline.Pipeline([('filter', best_filter), ('clf', base_clf)])

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_pipe)

my_X = sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0))
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(my_X, y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=68.5872459412
Logloss=1.8722423589
Time=18.8261601925
Accuracy=0.520767405063


/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-venv-auto/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:106: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/afs/inf.ed.ac.uk/user/s11/s1145806/Documents/git/neukrill-venv-auto/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:106: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


## Logistic Regression

In [48]:
clf = sklearn.linear_model.LogisticRegression(random_state=42)

In [49]:
t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0)), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=748.569411993
Accuracy=0.470266350211
Logloss=2.16218685973


In [50]:
clf = sklearn.linear_model.LogisticRegression(random_state=42)

my_X = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100).fit_transform(XF.squeeze(0), y)

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(my_X), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=145.000400066
Accuracy=0.527030590717
Logloss=1.83400284889


In [55]:
base_clf = sklearn.linear_model.LogisticRegression(random_state=42)

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_clf)

my_X = sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0))
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(my_X, y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=123.477735996
Logloss=2.53232405225
Time=0.659505844116
Accuracy=0.448641877637


In [52]:
base_clf = sklearn.linear_model.LogisticRegression(random_state=42)
best_filter = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100)

base_pipe = sklearn.pipeline.Pipeline([('filter', best_filter), ('clf', base_clf)])

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_pipe)

my_X = sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0))
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(my_X, y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=21.4947209358
Logloss=2.04123557253
Time=1.12955403328
Accuracy=0.476331751055


## Linear SVC

In [56]:
clf = sklearn.svm.SVC(kernel='linear', probability=True, random_state=42)

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0)), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=337.150140047
Accuracy=0.452202004219
Logloss=2.14135559841


In [57]:
clf = sklearn.svm.SVC(kernel='linear', probability=True, random_state=42)

my_X = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100).fit_transform(XF.squeeze(0), y)

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(my_X), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=86.2040469646
Accuracy=0.553270042194
Logloss=1.7817047324


In [58]:
base_clf = sklearn.svm.SVC(kernel='linear', probability=True, random_state=42)

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_clf)

my_X = sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0))
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(my_X, y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=1202.48394012
Logloss=2.14906265613
Time=122.273081064
Accuracy=0.434665084388


In [59]:
base_clf = sklearn.svm.SVC(kernel='linear', probability=True, random_state=42)
best_filter = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100)

base_pipe = sklearn.pipeline.Pipeline([('filter', best_filter), ('clf', base_clf)])

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_pipe)

my_X = sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0))
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(my_X, y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=177.089457035
Logloss=1.91130093883
Time=34.1300561428
Accuracy=0.488528481013


## Non-linear SVC

one-vs-one

In [60]:
clf = sklearn.svm.SVC(probability=True, random_state=42)

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0)), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=518.949584007
Accuracy=0.494857594937
Logloss=1.82661110623


In [61]:
clf = sklearn.svm.SVC(kernel='rbf', probability=True, random_state=42)

my_X = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100).fit_transform(XF.squeeze(0), y)

t0 = time.time()
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(
    sklearn.preprocessing.StandardScaler().fit_transform(my_X), y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

print("Accuracy={}".format(clf.score(X_test, y_test)))
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, clf.predict_proba(X_test))))

Time=121.488356829
Accuracy=0.522679324895
Logloss=1.70179310087


In [62]:
base_clf = sklearn.svm.SVC(kernel='rbf', probability=True, random_state=42)

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_clf)

my_X = sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0))
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(my_X, y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=446.839751005
Logloss=1.85225221785
Time=184.890357971
Accuracy=0.50039556962


In [63]:
base_clf = sklearn.svm.SVC(kernel='rbf', probability=True, random_state=42)
best_filter = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_classif, k=100)

base_pipe = sklearn.pipeline.Pipeline([('filter', best_filter), ('clf', base_clf)])

hier_clf = neukrill_net.stacked.HierarchyClassifier(marked_taxonomy, base_pipe)

my_X = sklearn.preprocessing.StandardScaler().fit_transform(XF.squeeze(0))
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(my_X, y, test_size=0.5, random_state=42)

t0 = time.time()
hier_clf.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print("Time={}".format(total))

t0 = time.time()
p = hier_clf.predict_proba(X_test)
print("Logloss={}".format(sklearn.metrics.log_loss(y_test, p)))
print("Time={}".format(time.time()-t0))

print("Accuracy={}".format(sklearn.metrics.accuracy_score(y_test,np.argmax(p,1))))

Time=128.495127916
Logloss=1.79123650959
Time=55.6328210831
Accuracy=0.507318037975
